# 导包，设置根目录

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re
import pickle

from collections import Counter
from itertools import chain
import datetime

# 更改工作目录为当前项目根目录
import sys
import os
os.chdir(os.path.dirname(os.path.dirname(sys.path[0])))
print(os.getcwd())

C:\workfile\python\Log-diagnosis


C:\Users\jinca\anaconda3\envs\Log_diagnosis_python\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\jinca\anaconda3\envs\Log_diagnosis_python\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
word_df = pd.read_csv('./pre_contest/dataset/word_label_df.txt',sep='\t',index_col=0)
word_df.groupby('label').size()

label
0    1460
1    3016
2    7731
3    2214
dtype: int64

In [3]:
log_wash_str_df = pd.read_csv('pre_contest/dataset/tags_incomplete.txt',sep='\t',names=['word'])

In [5]:
log_str_list=list(word_df['log'])
label_list=list(word_df['label'])
log_wash_str_list=list(log_wash_str_df['word'])

# 统计log_str_list在log_wash_str_list中的词频向量

In [6]:
frequency_vector_list = []
tag=0
for word in log_wash_str_list:
    if tag%50==0:
        print(tag,datetime.datetime.now())
    pattern=re.compile(word)
    frequency_vector = [len(re.findall(pattern,log))  for log in log_str_list]
    frequency_vector_list.append(frequency_vector)
    tag+=1

0 2022-03-19 23:32:17.881595
50 2022-03-19 23:32:18.709384
100 2022-03-19 23:32:19.546739
150 2022-03-19 23:32:20.369219
200 2022-03-19 23:32:21.186961
250 2022-03-19 23:32:21.987720
300 2022-03-19 23:32:22.783973
350 2022-03-19 23:32:23.554204
400 2022-03-19 23:32:24.325432
450 2022-03-19 23:32:25.130163
500 2022-03-19 23:32:25.926917


In [10]:
frequency_vector_df=pd.DataFrame(frequency_vector_list)
frequency_vector_df=frequency_vector_df.T
frequency_vector_df.columns=log_wash_str_list
frequency_vector_df['label']=label_list
frequency_vector_df['sn']=word_df['sn']
frequency_vector_df['day']=word_df['day']
# frequency_vector_df.to_csv('pre_contest/v1p1/v1p1_frequency_vector_df.csv',sep=',',index=None)

In [11]:
frequency_vector_df

,asserted,asserted power supply,asserted power supply ps1/2 status,asserted add-in card riser2 card,asserted battery,asserted boot error boot error,asserted button,asserted button button_pressed,asserted button buttonmm ev,asserted button power button,...,unknown error,unknown sps fw health,unrecoverable ide device failure,unspecified,upper,version change,watchdog ipmi watchdog,label,sn,day
0,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,SERVER_10001,2020-05-01
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_10003,2020-03-28
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,SERVER_10008,2020-02-25
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_10008,2020-03-11
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,SERVER_10009,2020-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14416,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9991,2020-08-04
14417,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9991,2020-10-07
14418,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9993,2020-05-14
14419,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,SERVER_9998,2020-05-29


# 训练XGB模型

In [8]:
random.seed(0)
feature=np.array(frequency_vector_list).T
label=np.array(label_list)
val_mask = [random.random() < 0.2 for _ in range(len(feature))]
train_mask = [not xx for xx in val_mask]
val_feature = feature[val_mask]
val_label = label[val_mask]
train_feature = feature[train_mask]
train_label = label[train_mask]

In [9]:
train_data=xgb.DMatrix(train_feature,label=train_label)
train_feature=xgb.DMatrix(train_feature)
val_feature=xgb.DMatrix(val_feature)
# xgb模型参数
params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':4,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':6,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':1, # 随机采样训练样本
    'colsample_bytree':1,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.3,  # 如同学习率
    'seed':1000,
    'nthread':12,  #CPU线程数
    #'eval_metric':'auc'
}
xgb_model=xgb.train(params,train_data,num_boost_round=500)

[19:18:06] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:18:06] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [10]:
train_pred=xgb_model.predict(train_feature)
val_pred=xgb_model.predict(val_feature)

In [11]:
# 指标评估
def macro_f1(label,prediction)  -> float:

    """
    计算得分
    :param target_df: [sn,fault_time,label]
    :param submit_df: [sn,fault_time,label]
    :return:
    """

    weights =  [3  /  7,  2  /  7,  1  /  7,  1  /  7]
    macro_F1 =  0.
    for i in  range(len(weights)):
        TP =  np.sum((label==i) & (prediction==i))
        FP =  np.sum((label!= i) & (prediction == i))
        FN =  np.sum((label == i) & (prediction!= i))
        precision = TP /  (TP + FP)  if  (TP + FP)  >  0  else  0
        recall = TP /  (TP + FN)  if  (TP + FN)  >  0  else  0
        F1 =  2  * precision * recall /  (precision + recall)  if  (precision + recall)  >  0  else  0
        macro_F1 += weights[i]  * F1
        
        print('Task %d:\n Prcesion %.2f, Recall %.2f, F1 %.2f' % (i+1, precision, recall, F1))
        
    return macro_F1

In [12]:
macro_f1(train_label,train_pred)

Task 1:
 Prcesion 0.90, Recall 0.44, F1 0.59
Task 2:
 Prcesion 0.79, Recall 0.92, F1 0.85
Task 3:
 Prcesion 0.97, Recall 0.98, F1 0.98
Task 4:
 Prcesion 0.92, Recall 0.97, F1 0.95


0.7722862888862191

In [13]:
macro_f1(val_label,val_pred)

Task 1:
 Prcesion 0.55, Recall 0.21, F1 0.31
Task 2:
 Prcesion 0.71, Recall 0.84, F1 0.77
Task 3:
 Prcesion 0.94, Recall 0.97, F1 0.96
Task 4:
 Prcesion 0.88, Recall 0.94, F1 0.91


0.6188138651405464

# 查看特征重要性

In [14]:
total_gain_dict=xgb_model.get_score(importance_type='total_gain')
total_gain_dict=dict(sorted(total_gain_dict.items(),key=lambda item:item[1],reverse=True))
feature_names=list(frequency_vector_df.columns)
feature_importance_top_100_dict={}
i=0
for key in total_gain_dict:
    if i<100:
        feature_name=feature_names[int(key[1:])]
        feature_importance_top_100_dict[feature_name]=total_gain_dict[key]
        i+=1
    else:
        break

In [15]:
feature_importance_top_100_dict

{'memory': 19951.212890625,
 'processor': 7185.7158203125,
 'unknown': 3174.168701171875,
 'asserted': 1186.5206298828125,
 'deassert': 1118.5928955078125,
 'correctable ecc': 711.334716796875,
 'configuration error': 580.76123046875,
 'uncorrectable ecc': 531.5259399414062,
 'microcontroller/coprocessor': 517.9636840820312,
 'microcontroller': 312.8116760253906,
 'power supply': 302.77862548828125,
 'uncorrectable machine check exception': 300.6255798339844,
 'state asserted': 287.34228515625,
 'fa': 266.56622314453125,
 'oem': 256.1112060546875,
 'power state': 250.35397338867188,
 'transition to running': 228.94210815429688,
 'ierr': 216.44268798828125,
 'failure detected': 203.43896484375,
 'system boot initiated': 185.61181640625,
 'presence detected': 173.03060913085938,
 'chassis': 167.32034301757812,
 'initiated by power up': 164.3397674560547,
 'temperature': 160.04908752441406,
 'memory cpu': 157.2410888671875,
 'system acpi power state acpi': 144.64247131347656,
 'initiated 

# 对初赛测试集进行预测

In [19]:
preliminary_submit_dataset=pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset.txt',sep='\t')
test_log_str_list=list(preliminary_submit_dataset['log'])
test_word_lists = [word_tokenize(str(ith)) for ith in test_log_str_list]
test_frequency_vector_list = []
tag=0
for word in word_frequency:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    test_frequency_vector = [sum(np.array([re.sub(r'[^\w]','',str(item)).lower() for item in test_word_lists[xth]]) == word) for xth in range(len(test_word_lists))]
    test_frequency_vector_list.append(test_frequency_vector)
    tag+=1


0 2022-03-15 00:14:59.990387
100 2022-03-15 00:15:20.008418
200 2022-03-15 00:15:40.003317
300 2022-03-15 00:15:59.992554
400 2022-03-15 00:16:19.983751
500 2022-03-15 00:16:39.992823
600 2022-03-15 00:17:00.068920
700 2022-03-15 00:17:20.107527
800 2022-03-15 00:17:40.166019
900 2022-03-15 00:18:00.253310
1000 2022-03-15 00:18:20.299403
1100 2022-03-15 00:18:40.293634
1200 2022-03-15 00:19:00.275526
1300 2022-03-15 00:19:20.286810
1400 2022-03-15 00:19:40.246421
1500 2022-03-15 00:20:00.237720
1600 2022-03-15 00:20:20.252362


In [20]:
test_frequency_vector_df=pd.DataFrame(test_frequency_vector_list)
test_frequency_vector_df=test_frequency_vector_df.T
test_frequency_vector_df.columns=list(word_frequency.keys())
test_frequency_vector_df[['sn','fault_time']]=preliminary_submit_dataset[['sn','fault_time']]
# test_frequency_vector_df.to_csv('pre_contest/dataset/test_frequency_vector_df.txt',sep='\t')

In [168]:
test_frequency_vector_df

,asserted,power,system,memory,drive,initiated,deasserted,processor,boot,state,...,nic11,dimm031_status,nic13,dimm171_status,9120,dimm1b,memory_hot,00fc92f15f00,sn,fault_time
0,2,1,2,0,0,4,0,0,2,0,...,0,0,0,0,0,0,0,0,000d33b21436,2020-09-02 16:42:54
1,4,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,005c5a9218ba,2020-06-28 19:05:16
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0079283bde6e,2020-04-26 21:32:44
3,5,2,3,1,0,2,0,0,1,2,...,0,0,0,0,0,0,0,0,007bdf23b62f,2020-06-16 18:40:39
4,6,0,3,2,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,00a577a8e54f,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffbf46b4af21,2019-12-28 20:10:01
3007,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ffc229b6cd9a,2020-06-27 02:39:08
3008,2,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,ffd44698a52b,2020-01-21 15:46:56
3009,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fff73a9e5bd5,2020-03-01 22:43:43


In [185]:
test_frequency_vector_df=pd.read_csv('pre_contest/dataset/test_frequency_vector_df.txt',sep='\t')
train_data=xgb.DMatrix(feature,label=label)
xgb_model_V1=xgb.train(params,train_data,num_boost_round=500)
test_feature_temp=np.array(test_frequency_vector_list).T
test_feature=xgb.DMatrix(test_feature_temp)
preliminary_submit_dataset['prediction']=xgb_model.predict(test_feature)

[21:57:21] WARNING: ..\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:57:21] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [209]:
preliminary_submit_dataset_a=preliminary_submit_dataset[['sn','fault_time','prediction']]
preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))
preliminary_submit_dataset_a=preliminary_submit_dataset_a[['sn','fault_time','label']]
preliminary_submit_dataset_a.to_csv('pre_contest/dataset/preliminary_submit_dataset_a_V1.csv',sep=',',index=None)

C:\Users\jinca\AppData\Local\Temp\ipykernel_2032\2213533351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))


In [210]:
preliminary_submit_dataset_a

,sn,fault_time,label
0,000d33b21436,2020-09-02 16:42:54,3
1,005c5a9218ba,2020-06-28 19:05:16,2
2,0079283bde6e,2020-04-26 21:32:44,3
3,007bdf23b62f,2020-06-16 18:40:39,2
4,00a577a8e54f,2020-04-07 07:16:55,2
...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,2
3007,ffc229b6cd9a,2020-06-27 02:39:08,2
3008,ffd44698a52b,2020-01-21 15:46:56,2
3009,fff73a9e5bd5,2020-03-01 22:43:43,2
